# 编译与反编译

首先说明下本文的python特指官方的Cpython实现.

python外界的定义是解释型语言,其原理是将代码放入解释器,解释一句执行一句.然而实际上python的执行流程会比这个略微复杂一些.python源码到执行的中间会有一个步骤生成字节码,实际执行的其实是这个字节码.因此,如果想跳过第一步,我们可以对源码进行编译,编译为字节码后直接执行字节码.

## python的字节码

cpython实际上运行的是字节码,我们所写的python源文件之所以可以被执行,其原因是当python程序加载时,源文件会被先翻译为字节码.然后保存在同级文件的`__pycache__`文件夹下.下次执行是,如果`__pycache__`下的字节码文件和源码文件修改时间匹配一致就会直接执行`__pycache__`中缓存的字节码,不一致则会将对应的字节码替换.


python的字节码在旧版本中有两种,一种是常规的`.pyc`文件,其加载速度相对于之前的.py文件有所提高,而且还可以实现源码隐藏,以及一定程度上的反编译;另一种是经过优化的`.pyo`文件(相比于`.pyc`文件更小),也可以提高加载速度.

而现在则统一都是以`.pyc`作为后缀,优化的等级则显式的放在编译后文件的命名上.

### 编译模块到`.pyc`

编译到`.pyc`我们可以使用两个标准库提供的工具:

+ `py_compile`用于编译单独的源码文件

+ `compileall`用于将某个文件夹下的源码文件递归地进行编译

#### **单文件编译**

使用`py_compile`单独编译一个文件的话,可以使用命令行工具

```shell
python -m py_compile /path/to/需要生成.pyc的脚本.py
```
即可.

也可以在python脚本下使用,调用`py_compile`:

```python
py_compile.compile(file[, cfile[, dfile[, doraise]]])
```

其中
+ `file`,表示需要生成`.pyc`或`.pyo`文件的源码路径;
+ `cfile`，表示需要生成`.pyc`或`.pyo`文件的目标文件路径.它默认是以`.pyc`为扩展名的形如`xxxx.cpython-36.pyc`这样形式的字符串.此外，当且仅当所使用的解释器允许编译成.pyo文件,才能以`.pyo`结尾,这个后文说.
+ `dfile`,表示编译出错时,将报错信息中的名字`file`替换为`dfile`
+ `doraise`，设置是否忽略异常.若为`True`,则抛出`PyCompileError`异常;否则直接将错误信息写入`sys.stderr`.


类似的,编译整个文件夹可以使用命令行工具`compileall`

```shell
python -m compileall <dir>
```

常用的参数有`-b`表示写入它们源文件同级并且同名不同后缀

也可以在python脚本下使用,调用`compileall`:

```python
import compileall

compileall.compile_dir('Lib/', force=True)
```
compile_dir参数有:

+ `dir`目标文件夹
+ `maxlevels=10`文件夹最大递归深度
+ `ddir=None`检查ddir中二进制文件的时间戳,如果与目标文件不一致才会编译
+ `force=False`强制编译
+ `rx=None`使用`re`查找目标文件夹下要编译的文件
+ `quiet=0`是否输出编译过程
+ `legacy=False`字节码文件是否被写入它们源文件同级并且同名(python2的默认编译行为),它们可能覆盖由另一版本的Python创建的字节码文件
+ `optimize=-1`优化等级,
+ `workers=1`多进程编译


#### **编译优化**

使用命令行工具编译是否优化的标志位在python一级,使用`-O`表示优化等级为1,`-OO`表示优化等级为2级(相比1级去掉了文档字符串),经过优化的字节码文件会被默认的命名为`xxxx.cpython-36.opt-2.pyc`这样的形式.

单文件的1级优化编译

```shell
python -O -m py_compile /path/to/需要生成.pyo的脚本.py
```

文件夹下递归2级优化编译:

```shell
python -O -m compileall -b <path>
```

需要注意编译后的字节码文件并不会提高运行时的执行速度,只会提高模块的加载速度.

## 反编译

指望通过编译python源文件来反编译是不现实的,使用`uncompyle6`工具可以非常轻易的将`.pyc`反编译为源码`.py`文件.`uncompyle6`可以直接使用`pip`工具安装.

## 结语

python的编译功能看起来挺鸡肋的,但还是可以在一些特定环境下发挥用处的,比如:

+ 硬盘空间资源紧张的硬件上,像树莓派,结合`zipapp`,可执行文件可以被控制在一个很小的程度.
+ 吓唬外行,通常知道点python的也不会知道可以`编译`,编译后的字节码人类也是没法直接解读的,一定程度上可以防止源码泄漏
